# Parte 3 - Visualizzazioni e query
A seguire alcune osservazioni sui dati accompagnate da visualizzazioni.
Le query sono fatte attraverso lo SPARQL endpoint locale di **OpenLink Virtuoso**.

In [2]:
from SPARQLWrapper import SPARQLWrapper, CSV
from io import StringIO
import pandas as pd
import numpy as np
import plotly.express as px
from urllib.parse import quote, unquote

sparql = SPARQLWrapper("http://localhost:8890/sparql")
sparql.setReturnFormat(CSV)
prefixes = """
prefix sdo: <http://www.so.developersurvey.org/ontology/>
prefix sdr: <http://www.so.developersurvey.org/resource/>
prefix dbo: <http://www.dbpedia.org/ontology/>
prefix dbr: <http://www.dbpedia.org/resource/>
"""

In [3]:
def truncate_uri(uri):
    return uri.rsplit('/', 1)[-1]

def deurify(uri):
    return unquote(str(uri).replace("_"," ").replace("","?"))

## Distribuzione geografica del dataset

Iniziamo con un'analisi della provenienza geografica delle risposte. Mi aspetto di vedere una maggioranza di utenti Americani.
Estraggo dal grafo il conteggio dei SurveyRespondant del 2023 raggruppati per codice ISO

In [4]:
sparql.setQuery(prefixes+"""
SELECT ?iso (COUNT(DISTINCT ?answer) AS ?total)
WHERE {
    ?answer a ?type;
            sdo:isFromCountry ?country .
    ?country sdo:ISO_A3 ?iso .
    ?type rdfs:subClassOf* sdo:SurveyRespondant .
}
GROUP BY ?iso
ORDER BY desc(?total)
""")

res=sparql.query().convert()
df=pd.read_csv(StringIO(res.decode('utf-8')))

df.head(20)


URLError: <urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>

Poi uso plotly express e il dataset gapminder per generare una mappa in scala logaritmica della densità di risposte in confronto alla popolazione.

In [ ]:
gm = px.data.gapminder()
gm = gm.loc[gm.year == 2007].drop(['year', 'lifeExp','gdpPercap', 'iso_num'], axis=1)

merged = gm.merge(df, left_on='iso_alpha', right_on='iso')
merged['w_tot'] = (merged['total'] / merged['pop']) * 1000000
merged['log_scale'] = np.log10(merged['w_tot'])

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

logmin = merged.log_scale.min()
logmax = merged.log_scale.max()

wmin = round(merged.w_tot.min())
wmax = round(merged.w_tot.max())


fig = go.Figure(go.Choropleth(
    locations=merged['iso_alpha'],
    z=merged['log_scale'],
    text=merged['country'],
    colorscale='plasma',
    locationmode='ISO-3',
    hoverinfo = 'text',
    colorbar=dict(
        len=0.75,
        title='Answers per million people',
        x=0.9,
        tickvals=[logmin, logmax],
        ticktext=[wmin, wmax]
    )
))



fig.update_geos(projection_type='natural earth')
# Show the plot
fig.show()

Possiamo osservare che il campione è sbilanciato verso la userbase di SO, cioè i developer che capiscono l'inglese. Paesi come gli stati uniti e l'australia hanno una concentrazione di risposte esponenzialmente maggiore rispetto ad altri stati attivi nel campo come la Cina e il Giappone.

Con una query federata a dbpedia sarebbe possibile ripetere l'osservazione filtrando solo i paesi che hanno come una delle lingue principali l'inglese.

## Utilizzo linguaggi di programmazione tra sviluppatori di generazioni diverse.
Continuiamo visualizzando l'andamento delle lingue di programmazione, ma suddivisi in due categorie in base all'età superiore o inferiore ai 30 anni.
I risultati con un totale sotto 3000 sono stati omessi per rendere il grafico leggibile.

In [ ]:
import plotly.express as px
sparql.setQuery(prefixes+"""
SELECT ?year ?lang ?young (COUNT(DISTINCT ?answer) AS ?total) 
WHERE {
    ?answer a ?type;
            sdo:isFromSurvey ?survey;
            sdo:usesLanguage ?langUri;
            sdo:minAge ?age .
    ?survey sdo:year ?year .
    ?type rdfs:subClassOf* sdo:SurveyRespondant .
    ?langUri rdfs:label ?lang.
            
    BIND(?age < 30 AS ?young)
}
ORDER BY ?year ?total
""")
res=sparql.query().convert()
df=pd.read_csv(StringIO(res.decode('utf-8')))
df.lang = df.lang.apply(unquote)
df.young = df.young.apply(lambda x: 'young' if x==1 else 'old')
df = df[df.total >= 3000]

In [ ]:
fig = px.line(df, x='year', y='total', color='lang', animation_frame='young',
                labels={'total': 'Total', 'year': 'Year'},
                title='Total Responses by Year for Different Languages',
                log_y=True,
              hover_name='lang', line_shape="spline")

fig.update_xaxes(dtick=1)


fig.show()

Mentre fonti come gli indici Tiobe e PYPL, ci dicono che Python è la lingua più cercata su google, sembrerebbe che al livello pratico la maggior parte degli sviluppatori si ritrovi a usare JavaScript nel proprio lavoro, con python al secondo posto tra le lingue di programmazione vere e proprie.

In generale non sembra esserci una grande differenza nelle preferenze delle due demografie, tuttavia nel timeframe considerato possiamo osservare un aumento sostanziale delle risposte da parte di utenti sotto i 30 anni, mentre l'altra categoria è più stabile.

## Utilizzo sistemi operativi
Adesso mettiamo al confronto le famiglie di sistemi operativi nel corso degli anni con una divisione per colore in base il tipo di utente.

In [ ]:
sparql.setQuery(prefixes + """
SELECT ?year ?os ?type  (COUNT(DISTINCT ?answer) AS ?count)
WHERE {
    ?answer a ?type;
            sdo:usesOS ?os;
            sdo:isFromSurvey ?survey
    ?survey a sdo:StackOverflowSurvey;
            sdo:year ?year.
    ?type rdfs:subClassOf* sdo:SurveyRespondant.
}
ORDER BY ?year ?os ?type
""")

res_csv=sparql.query().convert()
res=pd.read_csv(StringIO(res_csv.decode('utf-8')))
cols = ['os','type']
res[cols] = res[cols].apply(lambda x: x.apply(deurify))
res['type'] = res['type'].replace('SurveyRespondant', 'Other')

In [ ]:
fig = px.bar(res, x="count", y="os", color="type", title="Utilizzo sistemi operativi", animation_frame='year',
             labels={'count': 'Count', 'os': 'Sistema'})
fig.update_layout(
    font = dict(
        family = "Arial",
        size=16
    )
)
fig.show()



Possiamo osservare come sebbene sia facile confrontare questi dati all'interno dello stesso anno, il confronto tra anni diversi risulta alquanto difficile, dato che il contesto di come sono stati raccolti cambia drasticamente.
In particolare il forte aumento delle risposte totali nel 2022, non dovuto solo a un aumento di utenza, ma al fatto che è diventato possibile scegliere più opzioni.